# 2.1 - Collect initial data

## Background
Read up on the documentation of the NHS PROMs data to prepare for this lesson:
- [PROMs Guide](https://github.com/dkapitan/jads-nhs-proms-hko/blob/master/references/nhs/proms_guide_v12.pdf?raw=true)
- [Data Dictionairy](https://github.com/dkapitan/jads-nhs-proms-hko/blob/master/references/nhs/proms_data_dictionary.pdf?raw=true?)

## Objectives
- How to collect data?
- How to handle missing values?
- How to manage memory usage?
- How to rename columns and variables?

## Python skills
- Use `request` library to download files from the internet
- Use `zipfile` to work with zipfiles
- Know difference between getting a `bytes`, `string` and a `file` object for reading and writing data
- How to do string manipulation for replacing columns names
    - How to use [`toolz.functoolz.compose`](https://toolz.readthedocs.io/en/latest/api.html#toolz.functoolz.compose) to compose functions to operate in series
- How to use pandas:
    - for combining downloaded data into one dataframe: [`pd.concat`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)
    - do some memory optimization: [`df.select_dtypes`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html), [`df.astype`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html), [`pd.to_numeric`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html)
    - use [`df.rename`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) for renaming

In [1]:
from io import BytesIO
import requests
import re
from zipfile import ZipFile
import pandas as pd
from toolz.functoolz import compose


CONDITION = 'Knee'
ORG = 'Provider'

def get_zipped_data_urls(condition, org):
    """
    Get zipped datafiles for 3 years for condition and org
    
    condition: choice of ['Knee', 'Hip']
    org: choice of ['Provider', 'CCG']
    """

    assert condition in ['Knee', 'Hip'], 'Condition is one of ["Knee", "Hip"]'
    assert org in ['Provider', 'CCG'], 'Condition is one of ["Provider", "CCG"]'
    return [
        f'https://github.com/dkapitan/jads-nhs-proms-hko/blob/master/data/external/data-pack-2018-19/{condition}%20Replacement%20{org}%201819.csv.zip?raw=true',
        f'https://github.com/dkapitan/jads-nhs-proms-hko/blob/master/data/external/data-pack-2017-18/{condition}%20Replacements%20{org}%201718.csv.zip?raw=true',
        f'https://github.com/dkapitan/jads-nhs-proms-hko/blob/master/data/external/data-pack-2016-17/{condition}%20Replacement%20{org}%201617.csv.zip?raw=true',
    ]

def clean_python_name(s):
    """
    https://gist.github.com/dkapitan/89ff20eeed38e6d9757fef9e09e23c3d
    Method to convert string to clean string for use
    in dataframe column names such :
        i) it complies to python 2.x object name standard:
           (letter|'_')(letter|digit|'_')
        ii) my preference to use lowercase and adhere
            to practice of case-insensitive column names for data
    Based on
    https://stackoverflow.com/questions/3303312/how-do-i-convert-a-string-to-a-valid-variable-name-in-python
    """
    import re

    # Remove leading characters until we find a letter or underscore, and remove trailing spaces
    s = re.sub('^[^a-zA-Z_]+', '', s.strip())

    # Replace invalid characters with underscores
    s = re.sub('[^0-9a-zA-Z_]', '_', s)

    return s.lower()

In [2]:
# getting a single file
urls = get_zipped_data_urls(CONDITION, ORG)
get_ = requests.get(urls[0])
with ZipFile(BytesIO(get_.content)) as zipfile:
  # print(file_.namelist())
    df = pd.read_csv(zipfile.open(zipfile.namelist()[0]),
                    na_values=['*'])
df.head()

,Provider Code,Procedure,Revision Flag,Year,Age Band,Gender,Pre-Op Q Assisted,Pre-Op Q Assisted By,Pre-Op Q Symptom Period,Pre-Op Q Previous Surgery,...,Knee Replacement Post-Op Q Walking,Knee Replacement Post-Op Q Standing,Knee Replacement Post-Op Q Limping,Knee Replacement Post-Op Q Kneeling,Knee Replacement Post-Op Q Work,Knee Replacement Post-Op Q Confidence,Knee Replacement Post-Op Q Shopping,Knee Replacement Post-Op Q Stairs,Knee Replacement Post-Op Q Score,Knee Replacement OKS Post-Op Q Predicted
0,ADP02,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,...,3,3,4,2,4,4,4,3,40.0,34.825797
1,ADP02,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,...,4,4,4,3,3,3,4,4,44.0,43.180367
2,ADP02,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,...,4,4,4,2,4,4,4,4,46.0,42.201863
3,ADP02,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,...,1,4,4,1,3,3,4,3,36.0,36.826899
4,ADP02,Knee Replacement,0,2018/19,NaN,NaN,1,0,3,2,...,2,3,3,2,2,3,2,1,28.0,33.434301


In [3]:
# assignment: write a loop to get all files
dfs = []
for url in get_zipped_data_urls('Knee', 'CCG'):
    get_ = requests.get(url)
    with ZipFile(BytesIO(get_.content)) as zipfile:
        dfs.append(
            pd.read_csv(zipfile.open(zipfile.namelist()[0]), na_values=['*']))
df = pd.concat(dfs)

In [4]:
# check we have 3 years of data
df['Year'].unique()

array(['2018/19', '2017/18', '2016/17'], dtype=object)

In [5]:
# inspect memory usage: orignal is 85.3 MB
df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136390 entries, 0 to 46540
Columns: 81 entries, Provider Code to Knee Replacement OKS Post-Op Q Predicted
dtypes: float64(8), int64(69), object(4)
memory usage: 85.3+ MB


In [6]:
# reduce memory: strings to categories
# see: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html
# see: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype('category')
    
# int64 to smallest, unsigned int
for col in df.select_dtypes(include='int64').columns:
    df[col] = pd.to_numeric(df[col], downcast='unsigned')
    
# float64 to smallest, unsigned int
for col in df.select_dtypes(include='float64').columns:
    df[col] = pd.to_numeric(df[col], downcast='float')
    
# memory footprint now less than a fifth: 16 MB
df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136390 entries, 0 to 46540
Columns: 81 entries, Provider Code to Knee Replacement OKS Post-Op Q Predicted
dtypes: category(4), float32(8), uint16(2), uint32(2), uint8(65)
memory usage: 15.9 MB


In [7]:
# assignment: common string operations
cols = df.columns
cols

Index(['Provider Code', 'Procedure', 'Revision Flag', 'Year', 'Age Band',
       'Gender', 'Pre-Op Q Assisted', 'Pre-Op Q Assisted By',
       'Pre-Op Q Symptom Period', 'Pre-Op Q Previous Surgery',
       'Pre-Op Q Living Arrangements', 'Pre-Op Q Disability', 'Heart Disease',
       'High Bp', 'Stroke', 'Circulation', 'Lung Disease', 'Diabetes',
       'Kidney Disease', 'Nervous System', 'Liver Disease', 'Cancer',
       'Depression', 'Arthritis', 'Pre-Op Q Mobility', 'Pre-Op Q Self-Care',
       'Pre-Op Q Activity', 'Pre-Op Q Discomfort', 'Pre-Op Q Anxiety',
       'Pre-Op Q EQ5D Index Profile', 'Pre-Op Q EQ5D Index',
       'Post-Op Q Assisted', 'Post-Op Q Assisted By',
       'Post-Op Q Living Arrangements', 'Post-Op Q Disability',
       'Post-Op Q Mobility', 'Post-Op Q Self-Care', 'Post-Op Q Activity',
       'Post-Op Q Discomfort', 'Post-Op Q Anxiety', 'Post-Op Q Satisfaction',
       'Post-Op Q Sucess', 'Post-Op Q Allergy', 'Post-Op Q Bleeding',
       'Post-Op Q Wound', 'Post-

In [8]:
cols

Index(['Provider Code', 'Procedure', 'Revision Flag', 'Year', 'Age Band',
       'Gender', 'Pre-Op Q Assisted', 'Pre-Op Q Assisted By',
       'Pre-Op Q Symptom Period', 'Pre-Op Q Previous Surgery',
       'Pre-Op Q Living Arrangements', 'Pre-Op Q Disability', 'Heart Disease',
       'High Bp', 'Stroke', 'Circulation', 'Lung Disease', 'Diabetes',
       'Kidney Disease', 'Nervous System', 'Liver Disease', 'Cancer',
       'Depression', 'Arthritis', 'Pre-Op Q Mobility', 'Pre-Op Q Self-Care',
       'Pre-Op Q Activity', 'Pre-Op Q Discomfort', 'Pre-Op Q Anxiety',
       'Pre-Op Q EQ5D Index Profile', 'Pre-Op Q EQ5D Index',
       'Post-Op Q Assisted', 'Post-Op Q Assisted By',
       'Post-Op Q Living Arrangements', 'Post-Op Q Disability',
       'Post-Op Q Mobility', 'Post-Op Q Self-Care', 'Post-Op Q Activity',
       'Post-Op Q Discomfort', 'Post-Op Q Anxiety', 'Post-Op Q Satisfaction',
       'Post-Op Q Sucess', 'Post-Op Q Allergy', 'Post-Op Q Bleeding',
       'Post-Op Q Wound', 'Post-

In [9]:
def shorten_name(s):
    return (s.replace('pre_op_q', 't0')
             .replace('post_op_q', 't1')
             .replace('knee_replacement', 'oks')
             .replace('hip_replacement', 'ohs')
           )

df.rename(columns=compose(clean_python_name, shorten_name))

,provider_code,procedure,revision_flag,year,age_band,gender,pre_op_q_assisted,pre_op_q_assisted_by,pre_op_q_symptom_period,pre_op_q_previous_surgery,...,knee_replacement_post_op_q_walking,knee_replacement_post_op_q_standing,knee_replacement_post_op_q_limping,knee_replacement_post_op_q_kneeling,knee_replacement_post_op_q_work,knee_replacement_post_op_q_confidence,knee_replacement_post_op_q_shopping,knee_replacement_post_op_q_stairs,knee_replacement_post_op_q_score,knee_replacement_oks_post_op_q_predicted
0,00C,Knee Replacement,0,2018/19,NaN,NaN,1,0,2,2,...,4,3,4,4,4,4,4,2,43.0,35.133457
1,00C,Knee Replacement,0,2018/19,NaN,NaN,1,0,2,2,...,3,2,3,1,2,3,2,3,29.0,26.031412
2,00C,Knee Replacement,0,2018/19,NaN,NaN,2,0,4,2,...,2,2,1,0,2,2,0,2,16.0,26.818594
3,00C,Knee Replacement,0,2018/19,NaN,NaN,2,0,2,2,...,4,3,1,1,3,4,3,3,37.0,33.841702
4,00C,Knee Replacement,0,2018/19,NaN,NaN,2,0,1,1,...,4,4,4,3,4,4,4,4,47.0,38.841248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46536,99Q,Knee Replacement,0,2016/17,80 to 89,2.0,2,0,1,2,...,2,3,3,1,2,3,2,2,27.0,NaN
46537,99Q,Knee Replacement,0,2016/17,80 to 89,2.0,2,0,2,2,...,4,4,4,3,4,4,4,4,47.0,41.237152
46538,99Q,Knee Replacement,0,2016/17,80 to 89,2.0,2,0,3,2,...,4,4,4,3,4,4,4,4,46.0,41.244987
46539,99Q,Knee Replacement,0,2016/17,80 to 89,2.0,2,0,2,2,...,4,4,4,3,4,4,4,4,47.0,40.299709


In [10]:
# explain how you can use parquet. File 6.3 MB
df.to_parquet(f'../data/interim/{CONDITION}-{ORG}.parquet')